# Adding New Model Elements

This notebook walks through adding in-memory elements via PyMBE and tracking the additions to support updating model repositories after a session.

In [1]:
import pymbe.api as pm
from importlib import resources as lib_resources

from uuid import uuid4

from pymbe.model import Element
from pymbe.model import MetaModel
from pymbe.model_modification import \
    build_from_classifier_pattern, new_element_ownership_pattern, build_from_feature_pattern, apply_covered_feature_pattern

from pymbe.metamodel import \
    get_more_general_types, derive_inherited_featurememberships

from pymbe.query.metamodel_navigator import get_effective_basic_name

## Open Library Data

The library data are organized around namespaces, where each namespace corresponds to a specific file in the library.

In [2]:
library_model = None

with lib_resources.path("pymbe.static_data", "KernelLibrary.json") as lib_data:
    library_model = pm.Model.load_from_post_file(lib_data)

c:\users\bjorn cole\documents\github\pymbe\src\pymbe\label.py:144: UserWarning: Cannot process Expression elements yet!
  warn(f"Cannot process {expression._metatype} elements yet!")
c:\users\bjorn cole\documents\github\pymbe\src\pymbe\label.py:144: UserWarning: Cannot process BooleanExpression elements yet!
  warn(f"Cannot process {expression._metatype} elements yet!")


In [3]:
library_model.ownedElement[0]._data

{'elementId': '9cb49b7c-9a0e-4c54-8f17-6c3e4bcd022c',
 'aliasIds': [],
 '@type': 'Namespace',
 'ownedRelationship': [{'@id': '555fbf28-2469-4cc3-b655-84353891ddc9'}],
 'isImpliedIncluded': False,
 '@id': '9cb49b7c-9a0e-4c54-8f17-6c3e4bcd022c'}

## New Package
The most basic element in a model for grouping things is the package. It holds a place within a modeling project to associate other model contents.

The new root-level Package also needs to be inside a Namespace and become an owned member via an OwningMembership.

In [4]:
package_model_namespace_data = {
    'aliasIds': [],
    'isImpliedIncluded': False,
    '@type': "Namespace",
    '@id': str(uuid4()),
    'ownedRelationship': []
}

In [5]:
package_model_data = {
    'name': "User Process Model",
    'isLibraryElement': False,
    'filterCondition': [],
    'ownedElement': [],
    'owner': {},
    '@type': "Package",
    '@id': str(uuid4()),
    'ownedRelationship': []
}

In [6]:
empty_model = pm.Model(elements={})
empty_model

<SysML v2 Model (0 elements)>

In [7]:
new_ns = Element.new(data=package_model_namespace_data,model=empty_model)

In [8]:
new_package = Element.new(data=package_model_data,model=empty_model)
new_package

User Process Model «Package»

In [9]:
new_element_ownership_pattern(
    owner=new_ns, ele=new_package, model=empty_model, member_kind="OwningMembership"
)

«OwningMembership» ([4f963b48-e908-40b9-829b-fc23ebfe1274 «Namespace»] ←→ [User Process Model «Package»])

### Connect New Model to Library

Reference the new model to the library so inheritance and redefinition can be connected to library elements.

In [10]:
empty_model.reference_other_model(library_model)

In [11]:
peform_ns = [library_model_ns
               for library_model_ns in library_model.ownedElement
               if library_model_ns.throughOwningMembership[0].declaredName == 'Performances'][0]

perform_eles = peform_ns.throughOwningMembership[0].throughOwningMembership

performance = None
    
for perform_ele in perform_eles:
    if perform_ele._metatype in ('Behavior'):
        if hasattr(perform_ele, "declaredName"):
            if perform_ele.declaredName == 'Performance':
                performance = perform_ele
                
performance

Performance «Behavior»

In [12]:
performance._data

{'elementId': '1b7f00e7-24fb-5ebc-8773-89ba88bc3d5d',
 'owningRelationship': {'@id': '5c3904b9-9ab4-4830-a40a-e988839031f1'},
 'aliasIds': [],
 '@type': 'Behavior',
 'ownedRelationship': [{'@id': 'e590b404-10f3-4b17-b297-bd37c0b65b38'},
  {'@id': '539ef00b-df25-4b6c-b2c0-82ca6b91193e'},
  {'@id': '6ec014f1-959c-45f8-ad78-2057fac61107'},
  {'@id': '08ff62d4-77d5-402c-9a4c-415a422b4c23'},
  {'@id': '1f33a079-2c05-4608-9cda-9aa46e48e8c0'},
  {'@id': '20b5d486-05cf-437a-820f-85427f09939f'},
  {'@id': 'be01a810-8b04-4b97-be95-77df974b72b2'},
  {'@id': 'ee5f694b-9281-4199-bcc0-22cfd0bc3f1f'},
  {'@id': '67299fd7-ef2d-4478-9175-5a30a1efacab'},
  {'@id': '6a4fd353-68c0-446b-b188-30f8c096899d'},
  {'@id': '0a0f8e21-41b1-4b63-8df1-fb23118bc844'},
  {'@id': 'b6d0c3fe-1f69-4725-8aba-01b153a5a635'}],
 'declaredName': 'Performance',
 'isSufficient': False,
 'isAbstract': True,
 'isImpliedIncluded': False,
 '@id': '1b7f00e7-24fb-5ebc-8773-89ba88bc3d5d'}

## New Performance

A new element to specialize the Performance library element is created.

In [13]:
new_performance = build_from_classifier_pattern(
    owner=new_package,
    name="New Process",
    model=empty_model,
    metatype="Behavior",
    superclasses=[performance],
    specific_fields={"ownedRelationship": []}
)

In [14]:
new_performance._derived

defaultdict(list,
            {'label': 'New Process «Behavior»',
             'reverseOwningMembership': [{'@id': '3fe66d19-81b6-4bd2-ba72-338058428beb'}],
             'throughSubclassification': [{'@id': '1b7f00e7-24fb-5ebc-8773-89ba88bc3d5d'}]})

In [15]:
new_package.throughOwningMembership

[New Process «Behavior»]

In [16]:
new_process = new_package.throughOwningMembership[0]
new_process

New Process «Behavior»

In [17]:
new_process._data

{'owningRelationship': {'@id': '2ff886b4-dc3f-43e8-83d0-1d3a94dc41d4'},
 'ownedRelationship': [],
 'aliasIds': [],
 'isImpliedIncluded': False,
 'elementId': '',
 'isSufficient': False,
 'isAbstract': False,
 'declaredName': 'New Process',
 'declaredShortName': '',
 'documentation': None,
 'owningMembership': None,
 'ownedIntersecting': None,
 'ownedDisjoining': None,
 'ownedDifferencing': None,
 'step': None,
 'ownedFeature': None,
 'membership': None,
 'importedMembership': None,
 'ownedSubclassification': None,
 'textualRepresentation': None,
 'unioningType': None,
 'ownedUnioning': None,
 'differencingType': None,
 'ownedElement': None,
 'ownedEndFeature': None,
 'feature': None,
 'ownedConjugator': None,
 'ownedMember': None,
 'member': None,
 'multiplicity': None,
 'ownedAnnotation': None,
 'output': None,
 'endFeature': None,
 'intersectingType': None,
 'featureMembership': None,
 'directedFeature': None,
 'ownedMembership': None,
 'owningNamespace': None,
 'ownedFeatureMembersh

In [18]:
new_process._derived

defaultdict(list,
            {'label': 'New Process «Behavior»',
             'reverseOwningMembership': [{'@id': '3fe66d19-81b6-4bd2-ba72-338058428beb'}],
             'throughSubclassification': [{'@id': '1b7f00e7-24fb-5ebc-8773-89ba88bc3d5d'}]})

In [19]:
get_more_general_types(new_process,0,20)

[Performance «Behavior», Occurrence «Class», Anything «Classifier»]

In [20]:
new_process._data

{'owningRelationship': {'@id': '2ff886b4-dc3f-43e8-83d0-1d3a94dc41d4'},
 'ownedRelationship': [],
 'aliasIds': [],
 'isImpliedIncluded': False,
 'elementId': '',
 'isSufficient': False,
 'isAbstract': False,
 'declaredName': 'New Process',
 'declaredShortName': '',
 'documentation': None,
 'owningMembership': None,
 'ownedIntersecting': None,
 'ownedDisjoining': None,
 'ownedDifferencing': None,
 'step': None,
 'ownedFeature': None,
 'membership': None,
 'importedMembership': None,
 'ownedSubclassification': None,
 'textualRepresentation': None,
 'unioningType': None,
 'ownedUnioning': None,
 'differencingType': None,
 'ownedElement': None,
 'ownedEndFeature': None,
 'feature': None,
 'ownedConjugator': None,
 'ownedMember': None,
 'member': None,
 'multiplicity': None,
 'ownedAnnotation': None,
 'output': None,
 'endFeature': None,
 'intersectingType': None,
 'featureMembership': None,
 'directedFeature': None,
 'ownedMembership': None,
 'owningNamespace': None,
 'ownedFeatureMembersh

In [21]:
new_process.feature

[self «Feature»,
 involvedObjects «Feature»,
 performers «Feature»,
 :>>isDispatch «Feature»,
 :>>dispatchScope «Feature»,
 enclosedPerformances «Step»,
 thisPerformance «Feature»,
 «Connector» ([self «Feature»] ←→ [thisPerformance «Feature»]),
 subperformances «Step»,
 portionOfLife «Feature»,
 self «Feature»,
 sameLifeOccurrences «Feature»,
 this «Feature»,
 «Connector» ([self «Feature»] ←→ [this «Feature»]),
 localClock «Feature»,
 suboccurrences «Feature»,
 withoutOccurrences «Feature»,
 predecessors «Feature»,
 successors «Feature»,
 immediatePredecessors «Feature»,
 immediateSuccessors «Feature»,
 timeEnclosedOccurrences «Feature»,
 timeCoincidentOccurrences «Feature»,
 spaceEnclosedOccurrences «Feature»,
 spaceTimeEnclosedOccurrences «Feature»,
 spaceTimeEnclosedPoints «Feature»,
 spaceTimeCoincidentOccurrences «Feature»,
 outsideOfOccurrences «Feature»,
 justOutsideOfOccurrences «Feature»,
 matingOccurrences «Feature»,
 innerSpaceDimension «Feature»,
 innerSpaceDimension <= 3 «

In [22]:
performance.ownedRelationship

[«Subclassification» ([Performance «Behavior»] ←→ [Occurrence «Class»]),
 «Disjoining» ([Performance «Behavior»] ←→ [Object «Structure»]),
 «OwningMembership» ([Performance «Behavior»] ←→ [b895045c-6a93-4988-b017-c4240e0ee65d «Documentation»]),
 «FeatureMembership» ([Performance «Behavior»] ←→ [self «Feature»]),
 «FeatureMembership» ([Performance «Behavior»] ←→ [involvedObjects «Feature»]),
 «FeatureMembership» ([Performance «Behavior»] ←→ [performers «Feature»]),
 «FeatureMembership» ([Performance «Behavior»] ←→ [:>>isDispatch «Feature»]),
 «FeatureMembership» ([Performance «Behavior»] ←→ [:>>dispatchScope «Feature»]),
 «FeatureMembership» ([Performance «Behavior»] ←→ [enclosedPerformances «Step»]),
 «FeatureMembership» ([Performance «Behavior»] ←→ [thisPerformance «Feature»]),
 «FeatureMembership» ([Performance «Behavior»] ←→ [«Connector» ([self «Feature»] ←→ [thisPerformance «Feature»])]),
 «FeatureMembership» ([Performance «Behavior»] ←→ [subperformances «Step»])]

## New Object

Create a basic Object that is connected back to the libarry

In [23]:
object_ns = [library_model_ns
               for library_model_ns in library_model.ownedElement
               if library_model_ns.throughOwningMembership[0].declaredName == 'Objects'][0]

object_eles = object_ns.throughOwningMembership[0].throughOwningMembership

obj = None
    
for object_ele in object_eles:
    if object_ele._metatype in ('Structure'):
        if hasattr(object_ele, "declaredName"):
            if object_ele.declaredName == 'Object':
                obj = object_ele

In [24]:
obj

Object «Structure»

## New Step
Add steps to our earlier new Performance.

In [25]:
new_step_type_1 = build_from_classifier_pattern(
        owner=new_package,
        name="Sub-Process 1",
        model=empty_model,
        metatype="Behavior",
        superclasses=[performance],
        specific_fields={"ownedRelationship": []}
    )

new_step_type_2 = build_from_classifier_pattern(
    owner=new_package,
    name="Sub-Process 2",
    model=empty_model,
    metatype="Behavior",
    superclasses=[performance],
    specific_fields={"ownedRelationship": []}
)

new_step_1 = build_from_feature_pattern(
    owner=new_performance,
    name="Step 1",
    model=empty_model,
    specific_fields={},
    feature_type=new_step_type_1,
    direction="",
    metatype="Step",
    connector_end=False,
)

new_step_2 = build_from_feature_pattern(
    owner=new_performance,
    name="Step 2",
    model=empty_model,
    specific_fields={},
    feature_type=new_step_type_2,
    direction="",
    metatype="Step",
    connector_end=False,
)

In [26]:
(new_step_1, new_step_2)

(Step 1: Sub-Process 1 «Step», Step 2: Sub-Process 2 «Step»)

In [27]:
new_performance.throughFeatureMembership

[Step 1: Sub-Process 1 «Step», Step 2: Sub-Process 2 «Step»]

In [28]:
self_feat = None
time_enclosed_feat = None

for new_perf_feature in new_performance.feature:
    if get_effective_basic_name(new_perf_feature) == "self":
        self_feat = new_perf_feature
    if get_effective_basic_name(new_perf_feature) == "timeEnclosedOccurrences":
        time_enclosed_feat = new_perf_feature

In [29]:
new_performance_exec = build_from_classifier_pattern(
    owner=new_package,
    name="New Process as Executed",
    model=empty_model,
    metatype="Behavior",
    superclasses=[new_performance],
    specific_fields={"ownedRelationship": []}
)

step_1_exec = build_from_classifier_pattern(
    owner=new_package,
    name="Sub-Process 1 as Executed",
    model=empty_model,
    metatype="Behavior",
    superclasses=[new_step_type_1],
    specific_fields={"ownedRelationship": []}
)

step_2_exec = build_from_classifier_pattern(
    owner=new_package,
    name="Sub-Process 2 as Executed",
    model=empty_model,
    metatype="Behavior",
    superclasses=[new_step_type_2],
    specific_fields={"ownedRelationship": []}
)

In [30]:
covered_self = apply_covered_feature_pattern(
    one_member_classifiers=[new_performance_exec],
    feature_to_cover=self_feat,
    type_to_apply_pattern_on=new_performance_exec,
    model=empty_model,
    new_types_owner=new_package,
    covering_classifier_prefix="Class to Cover ",
    covering_classifier_suffix="",
    redefining_feature_prefix="",
    redefining_feature_suffix="(Closed)",
)
covered_self

self(Closed): New Process as Executed «Feature»

In [31]:
covered_self._derived

defaultdict(list,
            {'label': 'self(Closed): New Process as Executed «Feature»',
             'reverseFeatureMembership': [{'@id': '0bfdf0c0-0637-446c-86c4-b003143c8e36'}],
             'throughFeatureTyping': [{'@id': '0bfdf0c0-0637-446c-86c4-b003143c8e36'}]})

In [32]:
covered_self.reverseFeatureMembership

[New Process as Executed «Behavior»]

In [33]:
covered_time_enclosed = apply_covered_feature_pattern(
    one_member_classifiers=[new_performance_exec, step_1_exec, step_2_exec],
    feature_to_cover=time_enclosed_feat,
    type_to_apply_pattern_on=new_performance_exec,
    model=empty_model,
    new_types_owner=new_package,
    covering_classifier_prefix="Class to Cover ",
    covering_classifier_suffix="",
    redefining_feature_prefix="",
    redefining_feature_suffix="(Closed)",
)

In [37]:
covered_time_enclosed.throughFeatureTyping[0]._derived

defaultdict(list,
            {'label': 'Class to Cover timeEnclosedOccurrences «Behavior»',
             'reverseOwningMembership': [{'@id': '3fe66d19-81b6-4bd2-ba72-338058428beb'}],
             'throughUnioning': [{'@id': '0bfdf0c0-0637-446c-86c4-b003143c8e36'},
              {'@id': '765f6edd-ebaf-430f-ae05-fcc657e5c125'},
              {'@id': 'f4431126-8fac-499b-972a-5b66d8f4534f'}],
             'reverseFeatureTyping': [{'@id': '508ae7ad-5e2c-4e42-9c40-faa72b13f4e2'}]})

In [38]:
covered_time_enclosed.throughFeatureTyping[0].throughUnioning

[New Process as Executed «Behavior»,
 Sub-Process 1 as Executed «Behavior»,
 Sub-Process 2 as Executed «Behavior»]

## New OwningMembership

The OwningMembership, of which the OwningFeatureMembership is a sub-metatype, is what is used to organize the model hierarchy. Connecting two elements with this relationship should also updated derived fields such as owner and ownedElements.

In [34]:
new_om_source = new_package
new_om_target = new_classifier
new_om_source_id = new_package._id
new_om_target_id = new_classifier._id
om_data = {
    'isLibraryElement': False,
    'documentation': [],
    'source': [{'@id': new_om_source_id}],
    'target': [{'@id': new_om_target_id}],
    'owningRelatedElement': {'@id': new_om_source_id},
    'ownedRelatedElement': [{'@id': new_om_target_id}],
    'relatedElement': [{'@id': new_om_source_id}, {'@id': new_om_target_id}],
    '@type': "OwningMembership",
    '@id': str(uuid4())
}
new_om = Element.new(data=om_data,model=sysml_model)
new_om
(new_om_source_id, new_om_target_id)


NameError: name 'new_classifier' is not defined

In [ ]:
new_om

In [ ]:
new_package.ownedElement

### Derive Properties Due to Relationship

On the two ends of the relationship are fields that need to have fields updated

In [ ]:
new_package._data.update(
    {'ownedElement': new_package._data['ownedElement'] + [{'@id': new_om_target_id}]
    }
)
new_package.resolve()

new_classifier._data.update(
    {'owner': {'@id': new_om_source_id}
    }
)
new_classifier.resolve()

In [ ]:
new_package._data

In [ ]:
new_package.ownedElement

In [ ]:
new_classifier.owner

In [ ]:
new_package['ownedElement']